In this lecture we're going to address how you can bring multiple dataframe objects together, either by
merging them horizontally, or by concatenating them vertically. Before we jump into the code, we need to
address a little relational theory and to get some language conventions down. I'm going to bring in an image
to help explain some concepts.

![Venn Diagram](merging1.png)


In [6]:
#with that background lets see an example of how we would do this in pandas, where we would use the merge function
import pandas as pd

#first we create two dataframes, staff and students.
staff_df = pd.DataFrame([{'name': 'kelly', 'role': 'Director of HR'},
                        {'name': 'sally', 'role': 'COurse liason'},
                        {'name': 'james', 'role': 'Grader'}])
#lets now index staff by name
staff_df = staff_df.set_index('name')

#now lets create a student dataframe
student_df = pd.DataFrame([{'name':'james', 'school': 'Business'},
                          {'name': 'mike', 'school':'Law'},
                          {'name': 'sally', 'school': 'Engineering'}])

#and we will index this too by name
student_df = student_df.set_index('name')

#lets print out the dataframes
print(staff_df)
print(student_df)

                 role
name                 
kelly  Director of HR
sally   COurse liason
james          Grader
            school
name              
james     Business
mike           Law
sally  Engineering


In [7]:
#there is some overlap in this dataframe in that james and sally are both students and staff but mike and kelly are not
#importantly, both dataframes are indexed along the value we want to merge them on, whic is called name


In [8]:
#if we want the union of these, we would call merge() passing in the dataframe on the left and the dataframe on the right and telling merge that we want it to use in our join
#we want to use the left and right indices as the joining columns

pd.merge(staff_df, student_df, how= 'outer', left_index = True, right_index = True)

,role,school
name,,
james,Grader,Business
kelly,Director of HR,NaN
mike,NaN,Law
sally,COurse liason,Engineering


In [9]:
#we see ffrom the resulting dataframe that everyone is listed and since mike does not have a role and john doe not have a school, those cells are listed as missing values

#if we want to get the intersection, that is just those who are a student and a staff, we could set the how attribute to inner.
#again, we set both left and right indices to be true as the joining columns

pd.merge(staff_df, student_df, how = 'inner', left_index= True, right_index = True)

,role,school
name,,
sally,COurse liason,Engineering
james,Grader,Business


In [10]:
#we see that the resulting dataframe has only james and sally in it. now there are other two common use casses when merging dataframes,
#and both are examples of what we would call set addition. the first is when we would want to get a list of all staff regardless of whether they were students or not.
#but if they were student we would want to get there tudent details as well. To do this we would use a left join. it is important to note the order of dataframes in this function
#the first datatframe is the left dataframe and the second i the right dataframe.

pd.merge(staff_df, student_df, how = 'left', left_index= True, right_index= True)

,role,school
name,,
kelly,Director of HR,NaN
sally,COurse liason,Engineering
james,Grader,Business


In [11]:
#now we want a list of all the students and their roles if they were also staff 
#to do this we do a left join

pd.merge(staff_df, student_df, how = 'right', left_index= True, right_index= True)

,role,school
name,,
james,Grader,Business
mike,NaN,Law
sally,COurse liason,Engineering


In [13]:
# We can also do it another way. The merge method has a couple of other interesting parameters. First, you
# don't need to use indices to join on, you can use columns as well. Here's an example. Here we have a
# parameter called "on", and we can assign a column that both dataframe has as the joining column

# First, lets remove our index from both of our dataframes
staff_df = staff_df.reset_index()
student_df = student_df.reset_index()

# Now lets merge using the on parameter
pd.merge(staff_df, student_df, how='right', on='name')

,index_x,name,role,index_y,school
0,1.0,sally,COurse liason,2,Engineering
1,2.0,james,Grader,0,Business
2,NaN,mike,NaN,1,Law


In [14]:
# So what happens when we have conflicts between the DataFrames? Let's take a look by creating new staff and
# student DataFrames that have a location information added to them.
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR', 
                          'Location': 'State Street'},
                         {'Name': 'Sally', 'Role': 'Course liasion', 
                          'Location': 'Washington Avenue'},
                         {'Name': 'James', 'Role': 'Grader', 
                          'Location': 'Washington Avenue'}])
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 
                            'Location': '1024 Billiard Avenue'},
                           {'Name': 'Mike', 'School': 'Law', 
                            'Location': 'Fraternity House #22'},
                           {'Name': 'Sally', 'School': 'Engineering', 
                            'Location': '512 Wilson Crescent'}])

# In the staff DataFrame, this is an office location where we can find the staff person. And we can see the
# Director of HR is on State Street, while the two students are on Washington Avenue, and these locations just
# happen to be right outside my window as I film this. But for the student DataFrame, the location information
# is actually their home address.

# The merge function preserves this information, but appends an _x or _y to help differentiate between which
# index went with which column of data. The _x is always the left DataFrame information, and the _y is always
# the right DataFrame information.

# Here, if we want all the staff information regardless of whether they were students or not. But if they were
# students, we would want to get their student details as well.Then we can do a left join and on the column of
# Name

pd.merge(staff_df, student_df, how='left', on='Name')

,Name,Role,Location_x,School,Location_y
0,Kelly,Director of HR,State Street,NaN,NaN
1,Sally,Course liasion,Washington Avenue,Engineering,512 Wilson Crescent
2,James,Grader,Washington Avenue,Business,1024 Billiard Avenue


In [15]:
# From the output, we can see there are columns Location_x and Location_y. Location_x refers to the Location
# column in the left dataframe, which is staff dataframe and Location_y refers to the Location column in the
# right dataframe, which is student dataframe.

# Before we leave merging of DataFrames, let's talk about multi-indexing and multiple columns. It's quite
# possible that the first name for students and staff might overlap, but the last name might not. In this
# case, we use a list of the multiple columns that should be used to join keys from both dataframes on the on
# parameter. Recall that the column name(s) assigned to the on parameter needs to exist in both dataframes.

# Here's an example with some new student and staff data
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 
                          'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 
                          'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 
                          'Role': 'Grader'}])
student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 
                            'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 
                            'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 
                            'School': 'Engineering'}])

# As you see here, James Wilde and James Hammond don't match on both keys since they have different last
# names. So we would expect that an inner join doesn't include these individuals in the output, and only Sally
# Brooks will be retained.
pd.merge(staff_df, student_df, how='inner', on=['First Name','Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering


In [ ]:
# Joining dataframes through merging is incredibly common, and you'll need to know how to pull data from
# different sources, clean it, and join it for analysis. This is a staple not only of pandas, but of database
# technologies as well.

In [ ]:
# If we think of merging as joining "horizontally", meaning we join on similar values in a column found in two
# dataframes then concatenating is joining "vertically", meaning we put dataframes on top or at the bottom of
# each other

# Let's understand this from an example. You have a dataset that tracks some information over the years. And
# each year's record is a separate CSV and every CSV ofr every year's record has the exactly same columns.
# What happens if you want to put all the data, from all years' record, together? You can concatenate them.

In [ ]:
# Let's take a look at the US Department of Education College Scorecard data It has each US university's data
# on student completion, student debt, after-graduation income, etc. The data is stored in separate CSV's with
# each CSV containing a year's record Let's say we want the records from 2011 to 2013 we first create three
# dataframe, each containing one year's record. And, because the csv files we're working with are messy, I
# want to supress some of the jupyter warning messages and just tell read_csv to ignore bad lines, so I'm
# going to start the cell with a cell magic called %%capture

In [ ]:
%%capture
df_2011 = pd.read_csv("datasets/college_scorecard/MERGED2011_12_PP.csv", error_bad_lines=False)
df_2012 = pd.read_csv("datasets/college_scorecard/MERGED2012_13_PP.csv", error_bad_lines=False)
df_2013 = pd.read_csv("datasets/college_scorecard/MERGED2013_14_PP.csv", error_bad_lines=False)